# CNNでサムネイルを分類

In [ ]:
import tensorflow as tf
import os
import numpy as np
from PIL import Image
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from matplotlib import pylab as plt
%matplotlib inline

DIR = ['./main/', './sub/']
PREDICT_DIR = ['./predict_main/', './predict_sub/']
CATEGORY = ['メイン', 'サブ']

In [ ]:
def train_test():
    x = []
    y = []
    
    for i, directory in enumerate(DIR):
        files = os.listdir(directory)
        for file in files:
            image = Image.open(directory+file)
            image = image.convert('RGB')
            data = np.asarray(image)
            x.append(data)
            y.append(i)

    x = np.array(x)
    x = x.astype('float32')
    x = x / 255.0
    
    y = np.array(y)
    y = to_categorical(y)
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=(3, 3),
                     padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Conv2D(64, kernel_size=(3, 3),
                     padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=30)
    score = model.evaluate(x_test, y_test)
    print('loss:'+str(score[0]))
    print('acc:'+str(score[1]))
    
    predict = model.predict(x_test)
    count = 0
    for (pred, img) in zip(predict, x_test):
        plt.imsave(PREDICT_DIR[np.argmax(pred)]+str(count)+'.jpg', img)
        count += 1

In [ ]:
train_test()